# 5. Find largest observations (R)

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
pre_sample_file = "../preprocessed_data/pre_sample_2019-09-08.feather"

## Load data

In [3]:
pipelines <- feather::read_feather(pre_sample_file)
glimpse(pipelines)

Warning message:
“Coercing int64 to double”

Observations: 6,709
Variables: 17
$ OPERATOR_ID       <chr> "300", "300", "395", "402", "473", "473", "879", "9…
$ YEAR              <dbl> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 201…
$ NAME              <chr> "PLAINS PIPELINE, L.P.", "PLAINS PIPELINE, L.P.", "…
$ COMMODITY         <chr> "hvl", "non-hvl", "non-hvl", "hvl", "crude", "hvl",…
$ MILES             <dbl> 72.900, 0.000, 662.300, 1.550, 12.000, 0.000, 0.000…
$ AGE_UNKNOWN_MILES <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 5.155, 0.…
$ MILES_PRE_1940    <dbl> 0.000, 3.720, 0.600, 0.000, 0.000, 0.000, 0.265, 0.…
$ MILES_1940        <dbl> 0.000, 0.000, 15.000, 0.000, 0.000, 0.000, 0.000, 0…
$ MILES_1950        <dbl> 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 2.162, 0.…
$ MILES_1960        <dbl> 0.000, 0.000, 0.900, 0.000, 43.000, 0.000, 3.972, 0…
$ MILES_1970        <dbl> 0.000, 0.000, 1.500, 0.000, 4.000, 0.000, 5.181, 0.…
$ MILES_1980        <dbl> 0.000, 0.000, 18.500, 0.000, 0.000, 2.000, 0.000, 0…
$ MILES_1990      

## 5.1 Find largest observations

Ensure that we don't remove too many observations.

In [4]:
sample_size <- length(unique(pipelines$OPERATOR_ID))
print(sample_size)

[1] 701


In [5]:
offshore_size <- pipelines %>%
    group_by(OPERATOR_ID) %>%
    mutate(max_offshore = max(PERC_OFFSHORE)) %>%
    filter(max_offshore == 0) %>%
    ungroup() %>%
    {length(unique(.$OPERATOR_ID))}
print(offshore_size)

testit::assert(sample_size == offshore_size)

[1] 701


In [6]:
pipelines %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    top_n(1, MILES) %>%
    ungroup() %>%
    select(OPERATOR_ID, YEAR, NAME, COMMODITY, MILES) %>%
    top_n(10, MILES) %>%
    arrange(desc(MILES))

OPERATOR_ID,YEAR,NAME,COMMODITY,MILES
<chr>,<dbl>,<chr>,<chr>,<dbl>
31618,2018,ENTERPRISE PRODUCTS OPERATING LLC,hvl,7726.183
32109,2018,"ONEOK NGL PIPELINE, LLC",hvl,4531.480
2552,2015,COLONIAL PIPELINE CO,non-hvl,4500.920
1845,2015,"BUCKEYE PARTNERS, LP",non-hvl,4266.430
22610,2018,"MAGELLAN PIPELINE COMPANY, LP",non-hvl,3926.400
15674,2016,PLANTATION PIPE LINE CO,non-hvl,2564.730
31684,2005,CONOCOPHILLIPS PIPE LINE CO.,non-hvl,2558.000
11169,2017,"ENBRIDGE ENERGY, LIMITED PARTNERSHIP",crude,2290.000
32147,2013,MARATHON PIPE LINE LLC,non-hvl,2281.900


#### Largest for the sum of all types

In [7]:
unique(pipelines$COMMODITY)

[1] "hvl"     "non-hvl" "crude"

Check whether any operator ID has more than the maximum of five expected entries.

In [8]:
pipelines %>%
    group_by(YEAR, OPERATOR_ID) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    {table(.$n)}


   1    2    3 
4230  848  261 

Everything seems to be A-Okay.

In [9]:
largest_pipeline_operators <- pipelines %>%
    group_by(OPERATOR_ID, COMMODITY) %>%
    # Make sure we retrieve the latest name, no matter what year has to most extensive pipeline network
    arrange(desc(YEAR)) %>%
    mutate(NAME = first(NAME)) %>%
    group_by(OPERATOR_ID, YEAR) %>%
    summarize(NAME = first(NAME), TOTAL_MILES = sum(MILES)) %>%
    group_by(OPERATOR_ID) %>%
    top_n(1, TOTAL_MILES) %>%
    # Year as tie breaker
    top_n(1, YEAR) %>%
    ungroup() %>%
    arrange(desc(TOTAL_MILES))

head(largest_pipeline_operators, 200)

OPERATOR_ID,YEAR,NAME,TOTAL_MILES
<chr>,<dbl>,<chr>,<dbl>
31618,2018,ENTERPRISE PRODUCTS OPERATING LLC,8311.369
32109,2018,"ONEOK NGL PIPELINE, LLC",4756.610
31684,2005,PHILLIPS 66 PIPELINE LLC,4691.000
22610,2014,"MAGELLAN PIPELINE COMPANY, LP",4505.500
2552,2015,COLONIAL PIPELINE CO,4500.920
1845,2015,"BUCKEYE PARTNERS, LP",4298.290
18718,2010,SUNOCO PIPELINE L.P.,3466.740
22855,2004,"FLINT HILLS RESOURCES, LC",3051.000
19237,2008,"TE PRODUCTS PIPELINE COMPANY, LLC",2842.240


## 5.2 Save result

In [10]:
feather::write_feather(largest_pipeline_operators, paste0("../preprocessed_data/largest_companies_", Sys.Date(), ".feather"))

We also write the results to .csv, in order to add the parent company names. This column, we add as empty for now.

In [11]:
largest_pipeline_operators$PARENT <- NA

In [12]:
readr::write_csv(largest_pipeline_operators, paste0("../input/largest_companies_raw_", Sys.Date(), ".csv"))